In [ ]:
%reload_ext autoreload
%autoreload 2

# Logistic Regression Models

In [9]:
import autorootcwd
import json
import matplotlib.pyplot as plt
import os
import pandas as pd
from tqdm import tqdm

### Performances

In [ ]:
rows = []
DIR = "./results/logistic_lr_search"

for lr_scheduler in ['cosine', 'plateau', 'none']:
    for lr in [1e-05, 0.0001, 0.001, 0.01]:
        try:
            with open(f"{DIR}/lr-search_{lr_scheduler}_{lr}/val_metrics.json") as f:
                data = json.load(f)
                rows.append({
                    'lr_scheduler': lr_scheduler,
                    'lr': lr,
                    'accuracy': data['mean']['song_wise_acc'],
                    'root': data['mean']['root'],
                    'majmin': data['mean']['majmin'],
                    'third': data['mean']['third'],
                    'mirex': data['mean']['mirex'],
                    'acc-class': data['']
                })
        except:
            print(f"Failed to load {lr_scheduler} {lr}")
            pass

df = pd.DataFrame(rows)

In [ ]:
df

,lr_scheduler,lr,accuracy,root,majmin,third,mirex
0,cosine,0.00001,0.043714,0.112932,0.070033,0.072458,0.119268
1,cosine,0.00010,0.273072,0.493698,0.389092,0.390656,0.459411
2,cosine,0.00100,0.397829,0.620383,0.542229,0.541970,0.615230
3,cosine,0.01000,0.416999,0.638019,0.562913,0.563797,0.649865
4,plateau,0.00001,0.090979,0.189489,0.130495,0.132214,0.193630
5,plateau,0.00010,0.313479,0.541707,0.444475,0.445903,0.520074
6,plateau,0.00100,0.416932,0.642881,0.567921,0.567504,0.607393
7,plateau,0.01000,0.407415,0.635006,0.563191,0.564211,0.623632
8,none,0.00001,0.090979,0.189489,0.130495,0.132214,0.193630
9,none,0.00010,0.313479,0.541707,0.444475,0.445903,0.520074


### Training Graphs

In [ ]:
# Loop over directory of experiments
DIR = './results/logistic_lr_search/'
experiments = os.listdir(DIR)
for lr_scheduler in ['cosine', 'plateau', 'none']:
    for lr in [1e-5, 0.0001, 0.001, 0.01]:
        if lr_scheduler == 'none' and lr == 0.0001:
            continue
        exp = f'lr-search_{lr_scheduler}_{lr}'
        training_history = json.load(open(f'{DIR}/{exp}/training_history.json'))
        train_losses = training_history['train_losses']
        val_losses = training_history['val_losses']
        val_accuracies = training_history['val_accuracies']
        # Plot all 3, with acc having its own y-axis
        fig, ax1 = plt.subplots()
        ax2 = ax1.twinx()
        ax1.plot(train_losses, label='Training Loss', color='blue')
        # Multiply validations by 5 as they were only recorded every 5 epochs
        ax1.plot([5 * i for i in range(len(val_losses))], val_losses, label='Validation Loss', color='red')
        ax2.plot([5 * i for i in range(len(val_accuracies))], val_accuracies, label='Validation Accuracy', color='green')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss')
        ax2.set_ylabel('Accuracy')
        ax1.legend(loc='upper left')
        plt.title(f'{exp}')

# Re run for evaluation

In [4]:
import autorootcwd
from src.utils import get_split_filenames
from src.eval import evaluate_model
from src.data.dataset import FullChordDataset
from src.models.logistic_acr import LogisticACR
from src.models.cnn import CNN
import torch

In [6]:
# Load the best model
model = CNN(kernel_size=9, num_layers=5, channels=10, use_generative_features=False)
model.load_state_dict(torch.load('./results/cnn/cnn_k9_l5_c10/best_model.pth', map_location='mps', weights_only=True))
model.eval()

/Users/pierrelardet/.pyenv/versions/UG4Diss/lib/python3.8/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


CNN(
  (hmm_smoother): HMMSmoother()
  (temporal_cnn): Sequential(
    (0): Conv2d(1, 10, kernel_size=(9, 9), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(9, 9), stride=(1, 1), padding=same)
    (3): ReLU()
    (4): Conv2d(10, 10, kernel_size=(9, 9), stride=(1, 1), padding=same)
    (5): ReLU()
    (6): Conv2d(10, 10, kernel_size=(9, 9), stride=(1, 1), padding=same)
    (7): ReLU()
    (8): Conv2d(10, 10, kernel_size=(9, 9), stride=(1, 1), padding=same)
    (9): ReLU()
  )
  (freq_collapse): Conv2d(10, 36, kernel_size=(1, 216), stride=(1, 1))
  (classifier): Linear(in_features=36, out_features=170, bias=True)
)

In [12]:
def get_predictions(model, dataset):
    model.to('cpu')
    all_preds = []
    all_labels = []

    with torch.no_grad():  # Use no_grad to speed up inference
        for i in tqdm(range(len(dataset))):
            cqt, _, label = dataset[i]
            pred = model(cqt.unsqueeze(0))
            preds = torch.argmax(pred, dim=2)
            all_preds.append(preds[0])  # Keep as tensors
            all_labels.append(label)    # Keep as tensors

    # Concatenate all predictions and labels at the end
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)
    return all_preds, all_labels

all_preds, all_labels = get_predictions(model, val_dataset)

100%|██████████| 242/242 [06:19<00:00,  1.57s/it]


In [13]:
# overall accuracy
overall_accuracy = (all_preds == all_labels).float().mean()
overall_accuracy

tensor(0.5507)

In [7]:
train_filenames, val_filenames, test_filenames = get_split_filenames()
test_dataset = FullChordDataset(test_filenames, gen_layer=None)

# Evaluate the model on validation data
test_metrics = evaluate_model(model, test_dataset, batch_size=16)
test_metrics

100%|██████████| 16/16 [03:21<00:00, 12.62s/it]


{'mean': {'root': 0.7861017399423762,
  'majmin': 0.760402419261019,
  'mirex': 0.7898987247735245,
  'third': 0.7547091755130134,
  'seventh': 0.6248140921684602,
  'song_wise_acc': 0.6068177794262413},
 'median': {'root': 0.7917422484264613,
  'majmin': 0.7697265241147416,
  'mirex': 0.7960683223259043,
  'third': 0.7605957228043113,
  'seventh': 0.6062894874423463,
  'song_wise_acc': 0.5876923396107485},
 'frame_wise_acc': 0.608097208648358,
 'class_wise_acc_mean': 0.1801803593178583,
 'class_wise_acc_median': 0.0}